In [1]:
import torch
import numpy as np
import os
import json
import joblib
from torch.utils.data import Dataset,DataLoader
from itertools import repeat
import pandas as pd
import math
import torch.nn as nn
import torch.nn.functional as F
import pytorch_lightning as pl

/common/home/vk405/miniconda3/envs/Crossmdl/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# testing gpu
x_big = torch.randn(100000)
x_big_gpu = x_big.cuda()

In [4]:
x_big_gpu

tensor([-1.5254, -0.9146,  0.7285,  ...,  0.8204, -0.1264,  0.9060],
       device='cuda:0')

In [2]:
#utils

def get_vids(base_dir,split):
    trn_split = base_dir+split
    trn_idlst = []
    trn_vidlst = []

    f = open(trn_split,'r')
    for line in f:
        id_,vid = line.split('/')
        vid = vid.strip('\n')
        trn_idlst.append(id_)
        trn_vidlst.append(vid)
        #print(vid)
        #break
    f.close()
    return trn_idlst,trn_vidlst

    
def get_features(data_dir,split='val',feat_dir='/common/users/vk405/feat_csv/'):
    #feat_dir = data_dir
    splits_dir = data_dir+'splits/'
    if split == 'val':
        feat_split_dir = feat_dir+'val_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'val_list.txt')  
    elif split == 'train':
        feat_split_dir = feat_dir+'train_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'train_list.txt') 
    elif split == 'test':
        feat_split_dir = feat_dir+'test_frame_feat_csv/'  
        vid_num,vid_name = get_vids(splits_dir,'test_list.txt')
    else:
        raise NotImplementedError(f'unknown split: {split}')     
    feat_list = {}
    vid_dtls = []
    for num,name in zip(vid_num,vid_name):
        feat_loc = os.path.join(feat_split_dir, f'{num}/{name}/0001/')
        #import pdb;pdb.set_trace()
        if os.path.isdir(feat_loc):
            feat_files = feat_loc + os.listdir(feat_loc)[0]
            feat_list[name] = feat_files
            #feat_list.append(feat_files)
            vid_dtls.append((num,name))
        else:
            print(f"video : {num}/{name} not found")
    return feat_list,vid_dtls






def get_raw_labels(ids,annotns_file):

    label_info = {}
    with open(annotns_file) as json_file:
        annotns = json.load(json_file)
        print(annotns.keys())
        for _,vidname in ids:
            #import pdb;pdb.set_trace()
            if vidname in annotns['database']:
                #import pdb;pdb.set_trace()
                duration = annotns['database'][vidname]['duration']
                annot = annotns['database'][vidname]['annotations']
                labels = []
                #import pdb;pdb.set_trace()
                for segment_info in annot:
                    interval = segment_info['segment']
                    sent = segment_info['sentence']
                    labels.append((interval,sent,duration))

                label_info[vidname] = labels
            else:
                print(f"label for {vidname} not present")
    return label_info

def regress_labels(raw_labels):
    regress_labels = {}
    for key in raw_labels:
        new_labels = []
        for item in raw_labels[key]:
            rng,sent,vidlen = item
            mid = sum(rng)/2
            duration = rng[-1]-rng[0]
            mid_pred = (1/vidlen)*mid # location of mid-point w.r.t video length
            duration_pred = (1/vidlen)*duration
            new_labels.append(([mid_pred,duration_pred],sent))
        regress_labels[key] = new_labels
    return regress_labels
            
            
    
    
    


## Models

In [72]:
#!pip install transformers

def init_parameters_xavier_uniform(model):
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)

def scaled_dot(query, key, mask_key=None):  
    score = torch.matmul(query, key.transpose(-2, -1))
    score /= math.sqrt(query.size(-1))
    if mask_key is not None:
        score = score.masked_fill(mask_key, -1e18)  # Represents negative infinity
    return score      
            
def attend(query, key, value, mask_key=None, dropout=None):
    # TODO: Implement
    # Use scaled_dot, be sure to mask key
    #smax = nn.Softmax(-1)
    #import pdb;pdb.set_trace()
    score = scaled_dot(query,key,mask_key)  
    attention = F.softmax(score,dim=-1)
    if dropout is not None:#do = nn.Dropout(dropout)
        attention = dropout(attention)
    answer = torch.matmul(attention,value) 
    # Convexly combine value embeddings using attention, this should be just a matrix-matrix multiplication.
    return answer, attention



def split_heads(batch, num_heads):  
    (batch_size, length, dim) = batch.size()  # These are the expected batch dimensions.
    assert dim % num_heads == 0  # Assert that dimension is divisible by the number of heads.
    dim_head = dim // num_heads

    # No new memory allocation
    splitted = batch.view(batch_size, -1, num_heads, dim_head).transpose(1, 2)  
    return splitted  # (batch_size, num_heads, length, dim_head), note that now the last two dimensions are compatible with our attention functions. 




def merge_heads(batch):  
    (batch_size, num_heads, length, dim_head) = batch.size()  # These are the expected batch dimensions.

    # New memory allocation (reshape), can't avoid.
    merged = batch.transpose(1, 2).reshape(batch_size, -1, num_heads * dim_head)
    return merged  # (batch_size, length, dim)


class MultiHeadAttention(nn.Module):
    def __init__(self, dim, num_heads, dropout_rate=0.1):
        super().__init__()
        assert dim % num_heads == 0

        self.linear_query = nn.Linear(dim, dim)
        self.linear_key = nn.Linear(dim, dim)
        self.linear_value = nn.Linear(dim, dim)
        self.linear_final = nn.Linear(dim, dim)
        self.dropout = nn.Dropout(dropout_rate)

        self.num_heads = num_heads

    def forward(self, query, key, value, mask_key=None, layer_cache=None,
              memory_attention=False):
        """
        INPUT
          query: (batch_size, length_query, dim)
          key: (batch_size, length_key, dim)
          value: (batch_size, length_key, dim_value)
          mask_key: (*, 1, length_key) if queries share the same mask, else
                    (*, length_query, length_key)
          layer_cache: if not None, stepwise decoding (cache of key/value)
          memory_attention: doing memory attention in stepwise decoding?
        OUTPUT
          answer: (batch_size, length_query, dim_value)
          attention: (batch_size, num_heads, length_query, length_key) else
        """
        batch_size = query.size(0)

        query = self.linear_query(query)
        query = split_heads(query, self.num_heads)  # (batch_size, num_heads, -1, dim_head)

        def process_key_value(key, value):  # Only called when necessary.
            key = self.linear_key(key)
            key = split_heads(key, self.num_heads)
            value = self.linear_value(value)
            value = split_heads(value, self.num_heads)
            return key, value

        #import pdb;pdb.set_trace()
        if layer_cache is None:
            key, value = process_key_value(key, value)
        else:
            assert query.size(2) == 1  # Stepwise decoding
            
            if memory_attention:
                if layer_cache['memory_key'] is None:  # One-time calculation
                    key, value = process_key_value(key, value)
                    # (batch_size, num_heads, length_memory, dim)
                    layer_cache['memory_key'] = key
                    layer_cache['memory_value'] = value

                key = layer_cache['memory_key']
                value = layer_cache['memory_value']

            else:  # Self-attention during decoding
                key, value = process_key_value(key, value)
                assert key.size(2) == 1 and value.size(2) == 1
                
                # Append to previous.
                if layer_cache['self_key'] is not None:
                    key = torch.cat((layer_cache['self_key'], key), dim=2)
                    value = torch.cat((layer_cache['self_value'], value), dim=2)
                    
                 # (batch_size, num_heads, length_decoded, dim)
                layer_cache['self_key'] = key  # Recache.
                layer_cache['self_value'] = value
        # Because we've splitted embeddings into heads, we must also split the mask. 
        # And because each query uses the same mask for all heads (we don't use different masking for different heads), 
        # we can specify length 1 for the head dimension.
        if mask_key is not None:  
            mask_key = mask_key.unsqueeze(1)  # (batch_size, 1, -1, length_key)

        answer, attention = attend(query, key, value, mask_key, self.dropout)

        answer = merge_heads(answer)  # (batch_size, length_key, dim)
        answer = self.linear_final(answer)

        return answer, attention

class PositionwiseFeedForward(nn.Module):
    def __init__(self, dim, dim_hidden, drop_rate=0.1):
        super().__init__()
        self.w1 = nn.Linear(dim, dim_hidden)
        self.w2 = nn.Linear(dim_hidden, dim)
        self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
        self.drop1 = nn.Dropout(drop_rate)
        self.relu = nn.ReLU()
        self.drop2 = nn.Dropout(drop_rate)
    def forward(self, x):
        inter = self.drop1(self.relu(self.w1(self.layer_norm(x))))
        output = self.drop2(self.w2(inter))
        return output + x




class SinusoidalPositioner(nn.Module):
    def __init__(self, dim, drop_rate=0.1, length_max=5000):
        super().__init__()
        frequency = torch.exp(torch.arange(0, dim, 2) * -(math.log(10000.) / dim))  # Using different frequency for each dim
        positions = torch.arange(0, length_max).unsqueeze(1)
        wave = torch.zeros(length_max, dim)
        wave[:, 0::2] = torch.sin(frequency * positions)
        wave[:, 1::2] = torch.cos(frequency * positions)
        self.register_buffer('wave', wave.unsqueeze(0))  # (1, length_max, dim)
        self.dropout = nn.Dropout(drop_rate)
        self.dim = dim
        self.length_max = length_max
    def forward(self, x, step=-1):
        assert x.size(-2) <= self.length_max

        if step < 0:  # Take the corresponding leftmost embeddings.
            position_encoding = self.wave[:, :x.size(-2), :]
        else:  # Take the embedding at the step.
            position_encoding = self.wave[:, step, :]

        x = x * math.sqrt(self.dim)
        return self.dropout(x + position_encoding)




class TransformerEncoderLayer(nn.Module):

  def __init__(self, dim, num_heads, dim_hidden, drop_rate):
    super().__init__()
    self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
    self.self_attention = MultiHeadAttention(dim, num_heads, drop_rate)
    self.drop = nn.Dropout(drop_rate)
    self.feedforward = PositionwiseFeedForward(dim, dim_hidden, drop_rate)

  def forward(self, source, mask_source=None):
    # TODO: Implement
    #print(source.shape)
    normed = self.layer_norm(source)  
    # Apply layer norm on source

    attended, attention = self.self_attention(normed,normed,normed,mask_source)
    #None, None  # Apply self-attention on normed (be sure to use mask_source).
    attended = self.drop(attended) + source  
    # Re-write attended by applying dropout and adding a residual connection to source.
    return self.feedforward(attended), attention




class CrossAttentionLayer(nn.Module):
    def __init__(self,dim,num_heads,dim_hidden,drop_rate):
        super().__init__()
        self.layer_norm = nn.LayerNorm(dim, eps=1e-6)
        self.context_attention = MultiHeadAttention(dim, num_heads, drop_rate)
        self.drop = nn.Dropout(drop_rate)
        self.feedforward = PositionwiseFeedForward(dim, dim_hidden, drop_rate)
        
    def forward(self,target,memory,layer_cache=None):
        
        cross_attn_target = self.layer_norm(target)
        attended, attention = self.context_attention(cross_attn_target,memory,memory,layer_cache=layer_cache,memory_attention=True)
        
        attended = target + self.drop(attended)
        
        return self.feedforward(attended),attention



layer_cache = {'memory_key': None, 'memory_value': None, 'self_key': None, 'self_value': None}

In [64]:
positioner = SinusoidalPositioner(4, drop_rate=0., length_max=5)

In [147]:
mh_crossatn = CrossAttentionLayer(500,2,50,0.0)

source = torch.randn(1,500,500)
tgt = torch.randn(1,1,500)

In [149]:
output,_ = mh_crossatn(tgt,source)

In [150]:
output.shape

torch.Size([1, 1, 500])

In [79]:
total_params = lambda model : sum(p.numel() for p in model.parameters() if p.requires_grad)
#toomany params for a single layer.
total_params(mh_crossatn)

1002000

# Data-loading

In [3]:
#testing functions
data_dir = '/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'
annotns_file = data_dir+'annotations/youcookii_annotations_trainval.json'

splits_dir = data_dir+'splits/'
splits = ['test_list.txt','train_list.txt','val_list.txt']
trn_feats,trn_vids = get_features(data_dir,split='train')
label_info = get_raw_labels(trn_vids,annotns_file)
final_labels = regress_labels(label_info)

dict_keys(['database'])


In [8]:
trn_feats

{'Ysh60eirChU': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/Ysh60eirChU/0001/resnet_34_feat_mscoco.csv',
 'jpQBWsR3HHs': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/jpQBWsR3HHs/0001/resnet_34_feat_mscoco.csv',
 'QWXlKD-XGCQ': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/QWXlKD-XGCQ/0001/resnet_34_feat_mscoco.csv',
 '5E3kulJRzGY': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/5E3kulJRzGY/0001/resnet_34_feat_mscoco.csv',
 'arQZ2m0hMSU': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/arQZ2m0hMSU/0001/resnet_34_feat_mscoco.csv',
 'p1RgI4R8VX4': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/p1RgI4R8VX4/0001/resnet_34_feat_mscoco.csv',
 'frCFxOt9390': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/frCFxOt9390/0001/resnet_34_feat_mscoco.csv',
 'tasuUgO6m3c': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/tasuUgO6m3c/0001/resnet_34_feat_mscoco.csv',
 'KfAq4KRIVs4': '/common/users/vk405/feat_csv/train_frame_feat_csv/405/K

In [18]:
pd.read_csv(trn_feats['Ysh60eirChU']).values.shape

(499, 512)

In [103]:
# Dataset/loader

class YoucookDset(Dataset):
    def __init__(self,data_dir='/common/home/vk405/Projects/Crossmdl/Data/YouCookII/'\
        ,split='train',use_precomp_emb=True):
        self.split = split
        self.data_dir = data_dir
        self.use_precomp_emb = use_precomp_emb
        self.text_emb = None
        if self.split != 'test':
            self.annotns_file = data_dir+'annotations/youcookii_annotations_trainval.json'
        else:
            raise NotImplementedError(f"Split:{self.split},not yet correctly implemented")
        if self.use_precomp_emb:
            self.txt_emb = joblib.load(os.path.join(self.data_dir,'emb.joblib'))

        self.feat_locs,vids = get_features(self.data_dir,split=self.split)
        label_info = get_raw_labels(vids,self.annotns_file)
        self.final_labels = regress_labels(label_info)
        #(vid_id,seg_id)
        self.data = []
        #self.vid_len = []
        for key in final_labels:
            annot_len = len(final_labels[key])
            file_loc = self.feat_locs[key]
            segments = list(zip(repeat(key,annot_len),repeat(file_loc,annot_len),\
                    range(annot_len)))
            self.data.extend(segments)
    def __len__(self):
        return len(self.data)

    def __getitem__(self,idx):
        if self.use_precomp_emb:
            vidname,file_loc,seg_ind = self.data[idx]
            #import pdb;pdb.set_trace()
            #self.txt_emb[vidname][seg_ind],
            return pd.read_csv(file_loc).values.astype(np.float32),(self.txt_emb[vidname][seg_ind]).astype(np.float32),\
                np.array(self.final_labels[vidname][seg_ind][0],dtype=np.float32)
        else:
            raise NotImplementedError("not yet correctly implemented")

        

           



        

In [51]:
youcookdata = YoucookDset()

dict_keys(['database'])


In [104]:
vid_emb,txt_emb,label = youcookdata[0]

In [57]:
txt_emb.shape,label.shape

((768,), (2,))

In [105]:
youcookdl = DataLoader(youcookdata,batch_size=32,shuffle=True)

In [106]:
trn_feat,trn_wemb,trn_labels = next(iter(youcookdl))

In [ ]:
trn_feat.shape,trn_wemb.shape,trn_labels.shape
trn_wemb.unsqueeze_(1)


In [ ]:
trn_feat.transpose_(1,2)

In [ ]:
trn_wemb.transpose_(1,2)

In [110]:
trn_feat.shape,trn_wemb.shape

(torch.Size([32, 512, 499]), torch.Size([32, 768, 1]))

In [111]:
wrdcnn.weight.dtype,trn_feat.dtype

(torch.float32, torch.float64)

In [112]:
wrdcnn = nn.Conv1d(768, 100, 1, stride=1)
vidcnn = nn.Conv1d(512,100,1,stride=1)

trn_feat_red = vidcnn(trn_feat.float())
trn_wemb_red = wrdcnn(trn_wemb.float())

In [ ]:
trn_feat_red.transpose_(1,2),trn_wemb_red.transpose_(1,2)

In [118]:
trn_feat_red.shape

torch.Size([32, 499, 100])

In [117]:
trn_wemb_red.shape

torch.Size([32, 1, 100])

In [120]:
mlhattn = MultiHeadAttention(100,10,dropout_rate=0.0)

In [121]:
out,attn = mlhattn(trn_wemb_red,trn_feat_red,trn_feat_red)

In [122]:
out.shape

torch.Size([32, 1, 100])

In [125]:
total_params(mlhattn)

40400

In [184]:
from argparse import Namespace
hparams = Namespace(
    edim = 100,
    attnhdim = 50,
    nheads = 10,
    wrdim = 768,
    vidim = 512,
    hdim = 30,
    dropoutp=0.0

)

In [190]:
class CrossattnModel(pl.LightningModule):
    def __init__(self,hparams):
        super().__init__()
        self.save_hyperparameters(hparams)
        #self.hparams = hparams
        #import pdb;pdb.set_trace()
        self.attn = CrossAttentionLayer(self.hparams.edim,self.hparams.nheads,\
                           self.hparams.attnhdim,self.hparams.dropoutp)
        self.wrdcnn =  nn.Conv1d(self.hparams.wrdim, self.hparams.edim, 1, stride=1)
        self.vidcnn =  nn.Conv1d(self.hparams.vidim, self.hparams.edim, 1, stride=1)
        self.hid_layer = nn.Linear(self.hparams.edim,self.hparams.hdim)
        self.out_layer = nn.Linear(self.hparams.hdim,2)

    def forward(self,x):
        #keep this for inference
        out = self._model(x)
        return out
        
    def _model(self,x):
        vid_x,wrd_x = x
        #import pdb;pdb.set_trace()
        wrd_x = wrd_x.unsqueeze(1).transpose(1,2)
        vid_x = vid_x.transpose(1,2)
        #print(f"inside model, wrd_x:{wrd_x.shape},vi")
        tgt = self.wrdcnn(wrd_x.float()).transpose(1,2)
        src = self.vidcnn(vid_x.float()).transpose(1,2)
        attended,attn_score = self.attn(tgt,src)
        out = self.out_layer(F.relu(self.hid_layer(F.relu(attended))))
        return out


    def training_step(self,batch,batch_idx):
        #for tranining
        vid_feat,wrd_feat,labels = batch
        x_hat = self._model((vid_feat.float(),wrd_feat.float()))
        #import pdb;pdb.set_trace()
        loss = F.mse_loss(x_hat.squeeze().float(), labels.squeeze().float())
        print(f"inside train step, loss:{loss}")
        self.log("train_loss",loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer




In [191]:
model = CrossattnModel(hparams)

In [197]:
trainer = pl.Trainer(gpus=1)
youcookdl = DataLoader(youcookdata,batch_size=64,shuffle=True,num_workers=10)


GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


In [1]:
import numpy as np

In [194]:
#trainer.fit(model,youcookdl)

Epoch 3:  49%|████▉     | 160/324 [02:33<02:37,  1.04it/s, loss=0.0201, v_num=6]